In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import numpy as np

%load_ext autoreload
%autoreload 2
from data_loader import CharacterDataset

In [2]:
PATH='./telegram_fetch/texts/maria_1ua'
#PATH='./telegram_fetch/texts/elvina'

In [3]:
dataset = CharacterDataset(PATH)

data shape: (169675, 171)


In [4]:
x = dataset[0]
N = x.shape[-1]
print(f"Dimension size {N}")
print(f"Dataset length {len(dataset)}")

Dimension size 171
Dataset length 169675


In [77]:
class Encoder(nn.Module):
    def __init__(self, inp, out):
        super().__init__()
        self.inp = inp
        self.out = out
        N_layers = 2
        ratio = np.power(inp/out, 1/N_layers)
        mult = int(ratio*out)
        print(mult)
        
        self.dense1 = nn.Linear(self.inp, mult )
        self.bn1= nn.BatchNorm1d(mult)
        self.dense2 = nn.Linear(mult, out )
        
    def forward(self,x):
        x = F.relu(self.dense1(x))
        #x = self.bn1(x)
        x = self.dense2(x)
        return x
    
class RNN(nn.Module):
    def __init__(self, inp_d, hidden=300):
        super().__init__()
        self.enc = Encoder(inp=inp_d*2, out=hidden)
        self.dec = Encoder(out=inp_d, inp = hidden)
    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        x = F.softmax(x,dim=-1)
        return x


In [78]:
rnn = RNN(N, hidden=100)

def param_cnt(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"parameters count {param_cnt(rnn)}")
rnn

184
130
parameters count 117771


RNN(
  (enc): Encoder(
    (dense1): Linear(in_features=342, out_features=184, bias=True)
    (bn1): BatchNorm1d(184, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dense2): Linear(in_features=184, out_features=100, bias=True)
  )
  (dec): Encoder(
    (dense1): Linear(in_features=100, out_features=130, bias=True)
    (bn1): BatchNorm1d(130, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dense2): Linear(in_features=130, out_features=171, bias=True)
  )
)

In [116]:
import torch.optim as optim

criterion = nn.BCELoss()
opt = optim.SGD(rnn.parameters(), lr = 0.01, momentum=.99)

In [117]:
torch.cuda.is_available()

True

In [101]:
torch.cuda.set_device(0)
rnn.cuda()

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [118]:
import time
BATCH = 1
dataloader = DataLoader(dataset, batch_size=BATCH)

In [ ]:
start = time.time()
prev_output = torch.Tensor(np.zeros((BATCH,N)))
char = prev_output

for epoch in range(5):
    running_loss = 0.0
    for i, data in (enumerate(dataloader,0)):
        next_char = data
        next_char = torch.Tensor(next_char.float())
        
        opt.zero_grad()
        #char = char.cuda()
        inp = torch.cat((prev_output[:len(char)], char),1)
        outputs = rnn(inp)
        
        loss = criterion(outputs, next_char)
        loss.backward()
        opt.step()
        
        char = next_char.detach()
        prev_output = outputs.detach()
        running_loss += loss.item()
        if i % 20000 == 0:
            delta_t = time.time()-start
            print(f"[{epoch}, {i}] {np.round(delta_t,3)} loss: {running_loss/20000}")
            start = time.time()
            running_loss = 0.0
        

[0, 0] 0.008 loss: 1.7996544018387794e-06
[0, 20000] 16.261 loss: 0.01672755996523949
[0, 40000] 18.371 loss: 0.017582839579726322
[0, 60000] 19.335 loss: 0.01925735051558426
[0, 80000] 19.798 loss: 0.01756986014428938
[0, 100000] 19.846 loss: 0.01649588873477873
[0, 120000] 19.71 loss: 0.017195392981321674
[0, 140000] 19.784 loss: 0.016926406452417667
[0, 160000] 21.279 loss: 0.01710366651071812
[1, 0] 11.806 loss: 1.1668907478451729e-06
[1, 20000] 20.28 loss: 0.0156936457997036
[1, 40000] 22.069 loss: 0.016839345835651914
[1, 60000] 21.459 loss: 0.018657122431031407
[1, 80000] 20.657 loss: 0.01703789196292627
[1, 100000] 20.474 loss: 0.016031353062916406
[1, 120000] 20.718 loss: 0.016734721095841898
[1, 140000] 21.561 loss: 0.016490053541509408
[1, 160000] 20.319 loss: 0.01655510402564048
[2, 0] 10.516 loss: 1.1865735985338687e-06
[2, 20000] 20.797 loss: 0.01537713415319829
[2, 40000] 20.743 loss: 0.016540194617856475
[2, 60000] 21.23 loss: 0.01827123753972082
[2, 80000] 20.733 loss:

In [115]:
inital_char = ' '
prev_char = dataset._encoder.transform([[inital_char]]).todense()
inital_char = 'т'
curr_char = dataset._encoder.transform([[inital_char]]).todense()

curr_char= torch.Tensor(curr_char)
prev_char= torch.Tensor(prev_char)

for i in range(15):
    inp = torch.cat((prev_char, curr_char),1)
    char = rnn(inp)
    decoded_char = dataset._encoder.inverse_transform(char.detach())
    prev_char = curr_char
    curr_char = char
    print(decoded_char)

[[' ']]
[['н']]
[['и']]
[['т']]
[['и']]
[['л']]
[['и']]
[['л']]
[['а']]
[['л']]
[['и']]
[['л']]
[['и']]
[['л']]
[['и']]
